In [1]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import pathlib
import seaborn as sns
import pandas as pd
sns.set_context('talk')
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path_list = [
    "/storage/vbutoi/scratch/ESE/training/10_28_24_OCTA_FULLRESS_wAug",
]

df_list_1 = []
for path in path_list:
    try:
        dfc = rs.load_configs(
            path,
            properties=False,
        )
        df = rs.load_metrics(dfc)
        df_list_1.append(df)
    except Exception as e:
        print(e)

df = pd.concat(df_list_1)

In [3]:
def exp_name(path):
    exp_root = str(path).split("/")[-2]
    return exp_root

df.augment(exp_name)

In [ ]:
# We want to make sure that train goes before val in the phase
df['phase'] = df['phase'].astype('category')
df['version'] = df['version'].astype('category')
df['phase'].cat.reorder_categories(['train', 'val'], inplace=True)

In [5]:
# I only want seed = 40 for now
df = df[df['seed'] == 40]

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss',
    hue='exp_name',
    style='loss_func',
    col='phase',
    kind='line',
    height=12,
    facet_kws=dict(sharey=False),
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.12)
# Set the y axis to be between 0 and 1
# g.set(ylim=(0, 1000))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='dice_score',
    hue='exp_name',
    style='loss_func',
    col='phase',
    kind='line',
    height=12,
    # facet_kws=dict(sharey=False),
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.12)
# Set the y axis to be between 0 and 1
# g.set(ylim=(0, 1000))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='abs_area_estimation_error',
    hue='exp_name',
    style='loss_func',
    col='phase',
    kind='line',
    height=12,
    # facet_kws=dict(sharey=False),
)

# Set column spcig
g.fig.subplots_adjust(wspace=0.12)